In [ ]:
%pip install lightning
%pip install optuna
%load_ext tensorboard

In [13]:
!git clone https://github.com/lucas-battisti/custom-pytorch-lightning-module.git
!mv "custom-pytorch-lightning-module/modules.py" "modules.py"
!rm -rf "custom-pytorch-lightning-module"

!git clone https://github.com/lucas-battisti/pytorch-blocks.git
!mv "pytorch-blocks/blocks.py" "blocks.py"
!rm -rf "pytorch-blocks"

Cloning into 'custom-pytorch-lightning-module'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 25 (delta 11), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (25/25), 20.01 KiB | 853.00 KiB/s, done.
Cloning into 'pytorch-blocks'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 3), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (11/11), 17.19 KiB | 1.01 MiB/s, done.


In [ ]:
import lightning as L

from modules import RegressionModule as Reg
from blocks import LinearBlock, Conv1dBlock
from data import Custom_DataModule, CNN1D_Dataset

import torch
from torch import nn
from torch.nn import MaxPool1d, AvgPool1d
from torch.optim.lr_scheduler import ReduceLROnPlateau

import optuna
from optuna.integration import PyTorchLightningPruningCallback

import pandas as pd
from math import floor

import plotly

In [2]:
x = pd.read_csv("data/Dataframe_f.csv", index_col=0).astype('float32')
e = pd.read_csv("data/Dataframe_e_f.csv", index_col=0).astype('float32')
z = pd.read_csv("data/Dataframe_z.csv", index_col=0).astype('float32')

In [4]:

def default_training():
    L_IN = 16
    C_IN = 1
    
    dm = Custom_DataModule(xez = (x, e, z),
                           dataset_class=CNN1D_Dataset,
                           version='no', norm=False,
                           batch_size=200, num_workers=1)
    
    convblock1 = Conv1dBlock(in_channels=C_IN, out_channels=32,
                             kernel_size=3,
                             activation_function=nn.ReLU,
                             pooling_layer=MaxPool1d,
                             pooling_layer_args={"kernel_size": 2})
    
    L_convblock1 = convblock1.output_shape(L_IN)['L_out']
    
    convblock2 = Conv1dBlock(in_channels=32, out_channels=64,
                             kernel_size=2,
                             activation_function=nn.ReLU,
                             pooling_layer=MaxPool1d,
                             pooling_layer_args={"kernel_size": 2})
    
    L_convblock2 = convblock2.output_shape(L_convblock1)['L_out']
    
    dense_input = L_convblock2*64
    dense_intermed = int(dense_input*0.75)
    
    l1 = LinearBlock(in_features=dense_input, out_features=dense_intermed,
                     activation_function=nn.ReLU)
    
    l2 = LinearBlock(in_features=dense_intermed, out_features=dense_intermed,
                     activation_function=nn.ReLU)
    
    l3 = LinearBlock(in_features=dense_intermed, out_features=1)
    
    sequential = nn.Sequential(convblock1,
                               convblock2,
                               nn.Flatten(start_dim=1, end_dim=2),
                               l1,
                               l2,
                               l3)
    
    m = Reg(pytorch_module=sequential,
            loss_func=nn.MSELoss,
            optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3})
    
    return m, dm

In [ ]:
NUM_EPOCH = 500

m, dm = default_training()

trainer = L.Trainer(max_epochs=NUM_EPOCH,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)
trainer.fit(m, datamodule=dm)
trainer.test(m, datamodule=dm)

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    NUM_EPOCH = 300
    L_IN = 16
    C_IN = 1
    
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256, 512, 1024, 2048])
    
    dm = Custom_DataModule(xez = (x, e, z),
                           dataset_class=CNN1D_Dataset,
                           version='no', norm=False, #dataset_class kwargs
                           batch_size=batch_size, num_workers=1)
    
    n_conv_layers = trial.suggest_int("n_conv_layers", 1, 3)
    
    sequential = nn.Sequential()
    
    in_channels = [C_IN]
    out_channels = []
    kernel_size = []
    padding = []
    
    batch_norm_conv = trial.suggest_categorical("batch_norm_conv", [None, nn.BatchNorm1d])
    
    pool_type = trial.suggest_categorical("pool_type", [MaxPool1d, AvgPool1d])
    pool_kernel_size = []
    pool_padding = []
        
    length = [L_IN]
    
    
    
    for i in range(n_conv_layers):
        
        out_channels.append(trial.suggest_categorical("n_channels{}".format(i+1), [16, 32, 64, 128]))
        kernel_size.append(trial.suggest_int("kernel_size{}}".format(i+1), 1, length[i]))
        padding.append(trial.suggest_int("padding{}".format{i+1}, 0, int(floor(kernel_size[i]/2))))
        
        middle_length = length[i]+2*padding[i]-(kernel_size[i]-1)
        
        pool_kernel_size.append(trial.suggest_int("pool_kernel_size{}}".format(i+1), 1, middle_length))
        pool_padding.append(trial.suggest_int("padding{}".format{i+1}, 0, int(floor(pool_kernel_size[i]/2))))
        
        sequential = sequential.append(
            Conv1dBlock(in_channels=in_channels[i], out_channels=out_channels[i],
                        kernel_size=kernel_size[i], padding=padding[i],
                        norm_layer=batch_norm_conv, norm_layer_args={"num_features": out_channels[i]},
                        activation_function=nn.ReLU,
                        pooling_layer=pool_type, pooling_layer_args=
                        {"kernel_size":pool_kernel_size[i], "padding":pool_padding[i]})
        )
        

        length.append(int(sequential[i].output_shape(length[i])['L_out']))
        in_channels.append(out_channels[i])
    
    sequential = sequential.append(nn.Flatten(1, 2))
    
    first_input = out_channels[-1]*length[-1]
        
    n_dense_layers = trial.suggest_int("n_dense_layers", 1, 3)
    
    dense_layers_sizes = [first_input]
        
    batch_norm_dense = trial.suggest_categorical("batch_norm_dense", [None, nn.BatchNorm1d])
    
    for i in range(n_dense_layers):
        
        dense_layers_sizes.append(
            trial.suggest_int("dense_layer_size{}".format(i+1), 1, dense_layers_sizes[i]))
        
        sequential = sequential.append(
            LinearBlock(in_features=dense_layers_sizes[i], out_features=dense_layers_sizes[i+1],
                        norm_layer=batch_norm_dense, norm_layer_args={"num_features": dense_layers_sizes[i+1]},
                        activation_function=nn.ReLU))
        
    sequential = sequential.append(nn.Linear(dense_layers_sizes[-1], 1))
    
    lr = trial.suggest_loguniform("lr", 1e-6, 1)
    weight_decay = trial.suggest_uniform("weight_decay", 0, 1e-5)
    
    plateau_factor = trial.suggest_uniform("plateau_factor", 0, 1)
    
    m = Reg(pytorch_module=sequential,
            loss_func=nn.MSELoss,
            optimizer=torch.optim.Adam, optimizer_args={"lr": lr, "weight_decay": weight_decay},
            lr_scheduler=ReduceLROnPlateau,
            lr_scheduler_args={"mode": 'min', "factor": plateau_factor, "patience": 5, "threshold": 1e-3},
            tb_dir="CNN1D_no")
    
    trainer = L.Trainer(max_epochs=NUM_EPOCH,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1, callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss")], logger = True)
    trainer.fit(m, datamodule=dm)

    return_float = trainer.callback_metrics["val_loss"].item()

    trainer.test(m, datamodule=dm)

    return return_float